<a href="https://colab.research.google.com/github/LeNovice/Dissertation/blob/main/VGG_CadenceNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
USE_ORIGINAL = 0
loss = 'categorical_crossentropy'

In [2]:
import tensorflow_datasets as tfds
import numpy as np
#For plotting the dataset
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
#Data pipeline preparation
import tensorflow as tf
from tensorflow.keras import layers
#model building
from tensorflow.keras import models
import tensorflow.keras.utils as tfutils
import os
import copy

In [3]:
NUM_CLASSES = 10

DataSet = 'cifar10'
#'caltech101'
#'cifar10'
def num_samples_per_class(ds_train, get_top_10 = False, print_all = False):
    vals = np.unique(np.fromiter(ds_train.map(lambda x, y: y), int), return_counts=True)
    class_list = []
    class_hist = []
    for val,count in zip(*vals):
        if print_all==True:
            print(int(val), count)
        class_hist.append((val,count))
    if get_top_10 == True:
        sorted_tuple = sorted(class_hist, key=lambda t: t[-1], reverse=True)[:(NUM_CLASSES + 1)]    #+1 because we are going to remove "backround_google" i.e. 4
        class_list = [x for x,y in sorted_tuple]
    return class_list

def filter_fn(x, allowed_classes:list):
    allowed_classes = tf.constant(allowed_classes)
    isallowed = tf.equal(allowed_classes, tf.cast(x, allowed_classes.dtype))
    reduced_sum = tf.reduce_sum(tf.cast(isallowed, tf.float32))
    return tf.greater(reduced_sum, tf.constant(0.))

In [4]:
if DataSet == 'caltech101':
    ds_train, train_info = tfds.load(DataSet, split='test[0:90%]', as_supervised=True, with_info = True)
    ds_test = tfds.load(DataSet, split='train', as_supervised=True)
    ds_val = tfds.load(DataSet, split='test[90%:]', as_supervised=True) 
else:
    ds_train, train_info = tfds.load(DataSet, split='train[0:80%]', as_supervised=True, with_info = True)   #taking 0 to 80% for training
    ds_test = tfds.load(DataSet, split='test', as_supervised=True)        
    ds_val = tfds.load(DataSet, split='test[80%:]', as_supervised=True)                                     #taking data from 80% point to the end of the dataset (100%) for validation


In [5]:
class_list = num_samples_per_class(ds_train, get_top_10=True)
if DataSet == 'caltech101':
  class_list = [i for i in class_list if i != train_info.features['label'].str2int('background_google')]
  class_list.sort()
class_names = [train_info.features['label'].int2str(i) for i in class_list]

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [6]:
resized_ds_train = ds_train.filter(lambda x, y: filter_fn(y, class_list)) # as_supervised
resized_ds_test = ds_test.filter(lambda x, y: filter_fn(y, class_list))
resized_ds_val = ds_val.filter(lambda x, y: filter_fn(y, class_list))

In [7]:
#Hyperparameters
if DataSet=='caltech101':
    IMG_SIZE = 60
elif DataSet=='cifar10':
    IMG_SIZE = 32
NUM_CHANNELS = 3
BATCH_SIZE=128

input_shape = (IMG_SIZE,IMG_SIZE,NUM_CHANNELS)
#Relabelling to avoid issues. Note that human readability is reduced by this
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(class_list, dtype=tf.int64),
        values=tf.constant([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],  dtype=tf.int64)
    ),
    default_value= tf.constant(0,  dtype=tf.int64)
)

#This function will be used in the graph execution hence @tf.function prefix
@tf.function
def map_func(label):
    global class_list
    global loss
    mapped_label = table.lookup(label)
    if loss != 'sparse_categorical_crossentropy':
        mapped_label = tf.one_hot(indices=mapped_label, depth=NUM_CLASSES)
    print("Label = " + str(label) + "\t" + "Mapped Label = " + str(mapped_label))
    return mapped_label

#Preprocessing done as part of the graph
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

resize_layer = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
])

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

buffer_size = 30*NUM_CLASSES

#Preprocessing function which invokes above graphs
def prepare(ds, shuffle=False, augment=False, resize_only = False):
    global buffer_size
    global BATCH_SIZE
    

    # Resize and rescale all datasets.
    if resize_only==True:
        ds = ds.map(lambda x, y: (resize_layer(x), map_func(y)), num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = ds.map(lambda x, y: (resize_and_rescale(x), map_func(y)), num_parallel_calls=tf.data.AUTOTUNE)
    
    if shuffle:
        ds = ds.shuffle(buffer_size)
        
    # Batch all datasets.
    #ds = ds.batch(BATCH_SIZE)

    # Use data augmentation only on the training set.
    if augment:
        ds_aug = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.concatenate(ds_aug)

        
    # Use buffered prefetching on all datasets.
    return ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
resized_ds_train = prepare(resized_ds_train, augment=True)
resized_ds_test = prepare(resized_ds_test)
resized_ds_val = prepare(resized_ds_val)

Label = Tensor("label:0", shape=(), dtype=int64)	Mapped Label = Tensor("one_hot:0", shape=(10,), dtype=float32)


In [9]:
def num_samples_per_class_onehot(resized_ds_train, print_all=False):
    if loss != 'sparse_categorical_crossentropy':
        vals = np.unique(np.fromiter(resized_ds_train.map(lambda x, y: tf.argmax(y)), int), return_counts=True)
    else:
        vals = np.unique(np.fromiter(resized_ds_train.map(lambda x, y: y), int), return_counts=True)
    class_list = []
    class_hist = []
    for val,count in zip(*vals):
        if print_all==True:
            print(int(val), count)
        class_hist.append((val,count))
    class_hist.sort()
    return class_hist
#Post prepare function, all the labels will be converted to one hot encoders. In order to get class-wise distribution, we will need to convert each one hot encoder into its label (temporarily)
#We need a new function to handle it
class_hist = num_samples_per_class_onehot(resized_ds_train)


In [10]:
#reg = tf.keras.regularizers.L2(0.01)
reg = tf.keras.regularizers.L1L2(l1 =0.0, l2 = 0.1)
#reg = tf.keras.regularizers.L1L2(l1 =0.0, l2 = 0.0)
#beta_regularizer = 0.1
#gamma_regularizer = 0.1

model = models.Sequential()
kernel_size = (3,3)
pool_size = (2,2)
if USE_ORIGINAL == 1:
	display("Default quantization scheme")
	
	model.add(layers.Conv2D(32, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same', input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNELS)))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2D(32, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.MaxPooling2D(pool_size))
	model.add(layers.Dropout(0.1))
	
	model.add(layers.Conv2D(64, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2D(64, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.MaxPooling2D(pool_size))
	model.add(layers.Dropout(0.2))
	
	model.add(layers.Conv2D(128, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2D(128, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.MaxPooling2D(pool_size))
	model.add(layers.Dropout(0.3))
	
	model.add(layers.Flatten())
	model.add(layers.Dense(128, kernel_initializer='he_uniform', kernel_regularizer = reg))
	if 0:
		"""
		The converter quantizes batchnorm iff it follows a Conv2D layer. Hence we remove this BatchNorm layer (although it helps in accuracy).
		So we trade off accuracy for smaller model size
		"""
		model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Dropout(0.2))
	model.add(layers.Dense(NUM_CLASSES, kernel_regularizer = reg))
	model.add(layers.Softmax())
else:
	display("Quantization scheme experiments")
	
	model.add(layers.Conv2D(32, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same', input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNELS)))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2D(32, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.MaxPooling2D(pool_size))
	model.add(layers.Dropout(0.1))
	
	model.add(layers.Conv2D(64, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2D(64, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.MaxPooling2D(pool_size))
	model.add(layers.Dropout(0.2))
	
	model.add(layers.Conv2D(128, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2D(128, kernel_size, kernel_initializer='he_uniform', kernel_regularizer = reg, padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.MaxPooling2D(pool_size))
	model.add(layers.Dropout(0.3))
	
	model.add(layers.Flatten())
	model.add(layers.Dense(128, kernel_initializer='he_uniform', kernel_regularizer = reg))
	if 0:
		"""
		The converter quantizes batchnorm iff it follows a Conv2D layer. Hence we remove this BatchNorm layer (although it helps in accuracy).
		So we trade off accuracy for smaller model size
		"""
		model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Dropout(0.2))
	model.add(layers.Dense(NUM_CLASSES, kernel_regularizer = reg))
	model.add(layers.Softmax())

'Quantization scheme experiments'

In [11]:
def get_class_weights(class_hist):
    """
    Returns the class weights as a tf.Tensor. Class weights are inverse of the class frequencies
    Class frequencies are the number of samples of each class which we calculate in earlier steps
    """
    inv_freq = tf.convert_to_tensor([1.0/count for label, count in class_hist], dtype=tf.float32)
    return tfutils.normalize(inv_freq)


def weightedloss(y_true, y_pred, gamma, class_weight):
    """
    We assume that all arguments coming into this function are tf.Tensors type
    class_weights are basically alpha in focal loss paper
    """
    #ones = tf.convert_to_tensor(np.ones(shape=len(y_true)))
    a = tf.math.multiply(tf.math.pow(tf.math.subtract(1.0, y_pred), gamma), tf.math.log(y_pred))  #((1-pt)^gamma)log(pt)
    b = tf.math.multiply(-1.0, class_weight)                                                          #-alpha
    b = tf.math.multiply(b,a)    
    b = tf.math.multiply(b, y_true)
    return b
class WeightedLoss(tf.keras.losses.Loss):
    def __init__(self, gamma, class_weight=np.ones(shape=NUM_CLASSES, dtype=np.float32)):
        super().__init__()
        self.gamma = tf.convert_to_tensor(gamma)
        self.class_weight = tf.convert_to_tensor(class_weight, dtype=tf.float32)
    def call(self, y_true, y_pred):
        return weightedloss(y_true, y_pred, self.gamma, self.class_weight)

In [12]:
Learning_Rate = 1e-5

#tf.keras.optimizers.Adam(learning_rate=Learning_Rate)     #OR tf.keras.optimizers.SGD(learning_rate=Learning_Rate, momentum=0.9)
opt = tf.keras.optimizers.Adam(learning_rate=Learning_Rate)

In [13]:
###EITHER

#!pip install focal-loss
#from focal_loss import SparseCategoricalFocalLoss 
#model.compile( optimizer = opt, loss = SparseCategoricalFocalLoss(gamma=2), metrics=['accuracy'] )

In [14]:
###OR
model.compile( optimizer = opt, loss = loss, metrics=['accuracy'] )

In [15]:
###OR
#class_wts = get_class_weights(class_hist)
#display(class_wts)
#model.compile( optimizer = opt, loss = WeightedLoss(gamma=2.0), metrics=['accuracy'] )

In [16]:
#model.summary()

In [ ]:
resized_ds_train = resized_ds_train.batch(BATCH_SIZE)
resized_ds_val = resized_ds_val.batch(BATCH_SIZE)

h = model.fit( resized_ds_train, epochs=100, validation_data = resized_ds_val)

Epoch 1/100
625/625 [==============================] - 40s 52ms/step - loss: 111.9513 - accuracy: 0.1276 - val_loss: 103.5763 - val_accuracy: 0.2090
Epoch 2/100
625/625 [==============================] - 32s 52ms/step - loss: 95.8887 - accuracy: 0.1726 - val_loss: 88.1652 - val_accuracy: 0.2470
Epoch 3/100
625/625 [==============================] - 35s 56ms/step - loss: 81.1914 - accuracy: 0.2173 - val_loss: 74.3083 - val_accuracy: 0.2970
Epoch 4/100
625/625 [==============================] - 32s 51ms/step - loss: 68.2616 - accuracy: 0.2532 - val_loss: 62.3026 - val_accuracy: 0.3385
Epoch 5/100
625/625 [==============================] - 33s 52ms/step - loss: 57.1668 - accuracy: 0.2867 - val_loss: 52.0887 - val_accuracy: 0.3705
Epoch 6/100
625/625 [==============================] - 32s 52ms/step - loss: 47.7829 - accuracy: 0.3143 - val_loss: 43.4897 - val_accuracy: 0.3985
Epoch 7/100
625/625 [==============================] - 33s 52ms/step - loss: 39.9141 - accuracy: 0.3420 - val_loss: 

In [ ]:
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#Evaluation and confusion matrix creation:
from sklearn.metrics import classification_report, confusion_matrix

x_test = np.asarray(list(map(lambda x: x[0], tfds.as_numpy(resized_ds_test))))
y_test_orig = np.asarray(list(map(lambda x: x[1], tfds.as_numpy(resized_ds_test))))

In [ ]:
predictions = model.predict(x_test)

In [ ]:
if loss!='sparse_categorical_crossentropy':
    false_arr = np.full(shape=len(class_list), fill_value = False)
    #y_pred = np.empty(shape=y_test_orig.shape[-1])
    i=0
    for i, pred in enumerate(predictions):
        temp_arr = copy.deepcopy(false_arr)
        np.put(temp_arr, np.argmax(pred), True)
        if i==0:
            y_pred = copy.deepcopy(temp_arr)
        else:
            y_pred = np.vstack([y_pred, temp_arr])
    display(y_pred.shape)
else:
    y_pred = np.argmax(predictions, axis=1)


In [ ]:
display(y_test_orig.shape)
display(y_pred.shape)

In [ ]:
print('Confusion Matrix')
if loss != 'sparse_categorical_crossentropy':
    matrix = confusion_matrix(y_test_orig.argmax(axis=1), y_pred.argmax(axis=1))
else:
    matrix = confusion_matrix(y_test_orig, y_pred)
display(matrix)

# Print Classification Report
print('Classification Report')
print(classification_report(y_test_orig, y_pred, target_names=class_names))

In [ ]:
def ret_as_numpy():
    #test = tfds.load(DataSet, split='test', as_supervised=True)
    #test = prepare(test)
    #test = tfds.as_numpy(test)
    return tfds.as_numpy(resized_ds_test)

In [ ]:
test_as_np = ret_as_numpy()

In [ ]:
def evaluate_float_model(model, test):
    test_labels = []
    
    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for i, test_example in enumerate(test):
        test_labels.append(np.argmax(test_example[-1]))
        test_image = test_example[0]
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        #display(test_image.shape)
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        
        # Run inference.
        output = model(test_image, training=False)
        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = output.numpy()
        digit = np.argmax(output[0])
        prediction_digits.append(digit)
        
    print('\n')
    # Compare prediction results with ground truth labels to calculate accuracy.
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

In [ ]:
test_accuracy_Float = evaluate_float_model(model, test_as_np)

print('Float test_accuracy:', test_accuracy_Float)

Float checkpoint

In [ ]:
! pip install -q tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

#To make the whole model aware of quantization,
quantize_model = tfmot.quantization.keras.quantize_model

In [ ]:
q_aware_model = quantize_model(model)
#TODO: Check why this is not possible with Adam
opt = tf.keras.optimizers.SGD(learning_rate=Learning_Rate, momentum=0.9)
q_aware_model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics=['accuracy'])
q_aware_model.summary()

In [ ]:
h = q_aware_model.fit(resized_ds_train, epochs=5, validation_data = resized_ds_val)

In [ ]:
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

In [ ]:
def evaluate_model(interpreter, test):
    test_labels = []


    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    
    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for i, test_example in enumerate(test):
        test_labels.append(np.argmax(test_example[-1]))
        test_image = test_example[0]
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)
        
        # Run inference.
        interpreter.invoke()
        
        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)
        
    print('\n')
    # Compare prediction results with ground truth labels to calculate accuracy.
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

In [ ]:
#Models obtained from TfLiteConverter can be run in Python with Interpreter.
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
#Since TensorFlow Lite pre-plans tensor allocations to optimize inference, the user needs to call allocate_tensors() before any inference.
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter, test_as_np)

print('Quant TFLite test_accuracy:', test_accuracy)
#print('Quant TF test accuracy:', q_aware_model_accuracy)

In [ ]:
MODEL_DIR = "CadenceNet_Float"
model.save(MODEL_DIR, save_format="tf")

In [ ]:
!pip install -U tf2onnx==1.8.4
!python -m tf2onnx.convert --saved-model /content/CadenceNet_Float/ --output /content/CadenceNetOriginal_Float.onnx

In [ ]:
quant_file = "/content/CadenceNetOriginal_QAT.tflite"
open(quant_file, "wb").write(quantized_tflite_model)

In [ ]:

print("Float model in Mb: ", os.path.getsize("/content/CadenceNetOriginal_Float.onnx") / float(2**20))
print("Quantized model in Mb: ", os.path.getsize(quant_file) / float(2**20))
print("Float Model Accuracy: ", test_accuracy_Float)
print("Quantized Model Accuracy: ", test_accuracy)